### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2018 ###

# Megapolis #

#### Sam Berglin(berglin@wisc.edu), Aiden Song (aiden@cs.wisc.edu), Rachel Sowada(rsowada@wisc.edu), and Andrew Eng(ateng@wisc.edu)

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
    1. [First Model without Layout Mapping](#2.A.-First-Model-Without-Layout-Mapping)
    2. [Second Model with Layout Mapping:Zonning](#2.B.-Second-Model-With-Layout-Mapping:Zonning)
    3. [Third Model with Layout Mapping and Year Iteration](#2.C.-Third-Model-With-Layout-Mapping-and-Year-Iteration)
1. [Solution](#3.-Solution)
    1. [First Model without Layout Mapping](#3.A.-first-model)
    2. [Second Model with Layout Mapping](#3.B.-second-model)
    3. [Third Model with Layout Mapping and Year Iteration](#3.C.-third-model)
1. [Results and Discussion](#4.-Results-and-discussion)
  1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

City planning is the planning and designing of a city so that it can best control and facilitate economic growth, technological developments, and a sustainable population. Various aspects of city planning include the zoning of land for specific use only, regulating how space is used, what buildings can go on specific plots of land, how many fire and police stations within each block, etc.

One of the most well known city planning proposal is the [proposed new capital of Egypt in 2015](https://en.wikipedia.org/wiki/Proposed_new_capital_of_Egypt), a proposal to build a new city east of Cairo, Egypt. The major reason for undertaking of this project was to relieve the congestion in Cairo, which was one of the world's most crowded cities. In addition, games such as the SimCity series are primarily based on the concept of urban development and city planning. Whilst not realistic, these games provide a “sandbox” that allows players to create a magic city.

Creating these magical cities may seem like a simple task, however, when it comes to optimizing the most economically productive or most attractive city, there are many variables to consider. Land, tax rates, budgets, policies, and types of buildings are just a few that must be considered when planning out the “best” metropolis to build. 

For the most part, we will not be taking into account all of these concerns. Our project will instead be focusing on optimizing simplified magic cities before we try to tackle or discuss future additions or problems for more realistic models.

For now, however, there are ways we can model a simplified city and project its economic output or attractiveness for people to come and live in it. We can do this starting with the current plot of land or city. From there, we can evaluate its budget based on current economic output and population, and after putting a number to the year’s budget, then we can calculate the value each type of building will bring us. 

Certain buildings such as houses and schools don’t have much economic output. They do, however, hold a lot of attractive value for people who may want to move in. Similarly, some structures such as coal mines and manufacturing factories may not pull in a large permanent population, but these hold a ton of economic value to the current residents. We also take into consideration the importance of fire and police station for a community, hence optimize so that every block can be covered by a police and fire station.

<img src="http://www.urbandesigndirectory.com/sites/default/files/practices/Pablo%20Bullaude%203.jpg" height ="80%" width = "80%">


## 2.A. First Model Without Layout Mapping##

Our model can be represented as the mixed integer program (MIP),  
<br/>

<center>
$\begin{equation*}
\begin{aligned}
& \underset{x}{\text{maximize}}
& & x^T(P + A) \\
& \text{subject to}
& & x \geq p \cdot X_{min} \\
&&& B \geq x^TC \\
\end{aligned}
\end{equation*}$
</center>
<br/>
  
  In our model, $x = (x_1, x_2, ..., x_t)$ represents a vector of the number of each of the t types of buildings possible. Clearly, each of these values are integers.  
  
  $P, A, \text{and } C$ also represent similar quanitities, where $P = (p_1, p_2, ... p_n)$ and similar for $A$ and $C$. They are also t-dimensional vectors with the ith entry corresponding to the ith type of building. For a given building type i, $c_i$ represents the cost of constructing a building, $p_i$ represents the annual profit, and $a_i$ represents the attractiveness. In our simple model, we measure each of these for a building on a scale from -10 to 10. Negative numbers represent cost rather than profit, and "ugliness" rather than attractiveness (note: there cannot be a negative cost for constructing a building).  
  
  $X_{\text{min}}$ represents the minimum per person requirements for each building. Like $x$, it is t-dimensional for each of the minimum requirements for the t buildings. For clarity, we represent $X_{\text{min}}$ in number of buildings per 1,000 people. Furthermore, $p$ represents the number of people or population the city must be able to support.
  
  Since our decision variables (the number of buildings) must be positive integers, the problem is a MIP. In our model, we essentially are trying the maximize the annual "value" of a city given a certain budget for **completely** constructing it.

# 2.B. Second Model With Layout Mapping:Zonning  
  
  Obviously, choosing what buildings to build is not the entirety of how buildings are planned. We must choose where to place them, while still maintaining Level I's requirements.
  
  Consider the possible "world" to be an nxn board called $W$. $W[i,j]$ corresponds to the type of building built at location $[i,j]$ of the world. Given T types of possible buildings, we can have a one hot encoded vector $w_{i,j}=(0,0,..., 0, 1, 0, ..., 0)$ at each $W[i,j]$. The 1 at position $k$ of $w_{i,j}$ indicates that building type $k$ is built at $i,j$. We denote position $k$ of $w_{i,j}$ as $w_{i,j}^k$. We still have a similar $P$ and $A$ matrices. We represent our problem as below. We will explain the purpose of each constraint and notation afterwards.
  
<br/>
<center>
$\begin{equation*}
\begin{aligned}
& \underset{w_{i,j} \in W}{\text{maximize}}
& & \sum_{w_{i,j} \in W}t^T(P + A) \\
& \text{subject to}
& &  \forall b_{i,j} \in B, w_{i,j} \in W_\text{base} : w_{i,j} = b_{i,j} \space\space (A)\\
&&& \sum_{w_{i,j} \in W} w_{i, j} \geq X_\text{min} \cdot p \div 1000  \space\space (B)\\
&&& \sum_{w_{i,j} \in W} t^T C \leq B \space\space (C)\\
&&& \forall w_{i,j} \in W \sum_{1 = k}^{T} w_{i,j}^k \leq 1 \space\space (D)\\
&&& \forall w_{i,j} \in W_\text{inner} : w_{i,j} \leq w_{i-1,j} + w_{i,j-1} + w_{i+1,j} + w_{i,j+1} \space\space (E)\\
&&& \sum_{w_{i,j} \in W} (4w_{i,j} - w_{i-1,j} - w_{i+1,j} - w_{i,j+1} - w_{i,j-1}) \leq R \cdot \sum_{w_{i,j} \in W} w_{i,j} \space\space (F)\\
\end{aligned}
\end{equation*}$
</center>
<br/>
<br/>

  (A) This constraint ensures that the base city exists in the middle of the map and that the outside (or border) of the world is empty. The base city exists so that the next iteration can grow from it, and the border being empty aids in perimeter calculation in constraint F.
  
  (B) This constraint ensures that the population is suported by the proper number of buildings. It is the same as the toy example.
  
  (C) This constraint ensures that the budget is respected. It is the same as the toy example.
  
  (D) This constraint ensures that each vector $w_{i,j}$ can have at most one 1 and the remainder are zeros. This ensures that there is only one "type" of building assigned to each square of the world.
  
  (E) This is the first nontrivial constraint. It causes a phenomenon of zoning. Zoning is where certain areas of a community are reserved for certain types of buildings. For example, there can be residential zones or business zones. In order to implement this into our example, we ensure that each type of buildings must form a **continguous** area. This is represented in the constraint by not letting type $k$ of a building be built **unless** there is one of type $k$ surrounding it. $W_\text{inner}$ represents the inner world (i.e. the world without the border). We only sum over these squares since the border is defined to be zero.
  
  (F) This constraint builds upon constraint (E). As a whole, it prevents regions from being strangley shaped by ensuring a proper ratio of perimeter to area. The expression on the left calculates a T dimensional vector representing the perimeter of each of the contiguous building regions. The right side's summation calculates the area of the continguous building regions. The corresponding ratio is represented by $R$.

## 2.C.-Third-Model-With-Layout-Mapping-and-Year-Iteration

TEXT GOES HERE


---
## 3. Solution ##

We will first load all the packages and render all the helper functions 

In [83]:
using JuMP, Gurobi, Cbc
function xCoordinates(x1,x2)
    [x1
     x2
     x2
     x1
     x1
    NaN]
end

function yCoordinates(y1,y2)
    [y1
     y1
     y2
     y2
     y1
    NaN]
end
using Plots, Gurobi; plotly(xticks = nothing, yticks = nothing,grid = false) 
function graphMap(resultMatrix,area=[1,1,1,1,1,1,1])
   # loop through the position matrix, figure out the area of each, and then plot
    x_axis = 0; y_axis = 0;
    home = []; park = []; school = []; business = []; hospital = []; farm = [];empty = []
    buildingType = Dict(1 => business, 2 => hospital , 3 => park , 4 => farm, 5 => school, 6 => home, 7=> empty)
    arr = buildingType[1]    
    for i = 1:size(resultMatrix,1)
        max_side_length = 0
        for j = 1:size(resultMatrix,2)
            bType = resultMatrix[i,j];
            a = area[bType] # this gives the particular area of the building type
            side_length = sqrt(a)
            if side_length > max_side_length
                max_side_length = side_length
            end
            x1 = x_axis;
            x2 = x_axis + side_length;
            y1 = y_axis;
            y2 = y_axis + side_length;
            if bType == 6
                home = [
                    home 
                    [xCoordinates(x1,x2) yCoordinates(y1,y2)]
                ]
            elseif bType == 3
                park = [
                    park 
                    [xCoordinates(x1,x2) yCoordinates(y1,y2)]
                ]
            elseif bType == 5
                school = [
                    school
                    [xCoordinates(x1,x2) yCoordinates(y1,y2)]
                ]
            elseif bType == 2
                hospital = [
                    hospital
                    [xCoordinates(x1,x2) yCoordinates(y1,y2)]
                ]
            elseif bType == 1
                business = [
                    business
                    [xCoordinates(x1,x2) yCoordinates(y1,y2)]
                ]
            elseif bType == 4
                farm = [
                    farm
                    [xCoordinates(x1,x2) yCoordinates(y1,y2)]
                ]
            else
                empty = [
                    empty
                    [xCoordinates(x1,x2) yCoordinates(y1,y2)]
                ]
            end
            
            x_axis = x_axis + side_length + 0.3;
        end
        y_axis = y_axis + max_side_length + 0.3 ;
        x_axis = 0;
    end 
    plot(home[:,1], home[:,2],bg=:black,lw = 6,label="homes")
    plot!(business[:,1],business[:,2],lw=6,label=" business")
    plot!(school[:,1],school[:,2],lw = 6,label="school")
    plot!(park[:,1],park[:,2],lw = 6,label="park",leg= true)
    plot!(hospital[:,1],hospital[:,2], lw=6, label = "hospital")
    plot!(farm[:,1],farm[:,2],lw =6, label ="farm")
    plot!(empty[:,1], empty[:,2],lw=6, label=" ",linecolor=:black)
end

function getGraphMatrix(map)
    N = size(map,1); M = size(map,2); P = size(map,3)
    newMap = fill(0,(M,N))
    for i = 1:N
        for j = 1:M
            for k = 1:P
                if map[i,j,k] == 1
                    newMap[i,j] = k
                    break;
                end
            end
            if sum(map[i,j,k] for k = 1:P) == 0
                newMap[i,j] = 7
            end
        end
    end
    graphMap(newMap)
end

getGraphMatrix (generic function with 1 method)

## 3.A. First Model Without Layout Mapping

In [1]:
# data
building_types = ["business", "hospital", "park", "farm", "school", "home"]
num_types = length(building_types)
# we converted profits, and attractiveness on a scale from -10 to 10
cost = [20000,100000,3000,2000,60000,400]
profits = [9,-3,-1,6,-4,1]
attractiveness = [4,-1,3,-5,6,2]
min_num_building_per_1kPeople = [5,1,3,1,1,333]
;

In [2]:
using JuMP, Gurobi

m = Model(solver=GurobiSolver(OutputFlag=0))

# assume that we have population of 10,000 people
population = 10
# assume that we have a budget of 
budgets = 10000000

@variable(m, buildings[1:6],Int)
@constraint(m,var[i = 1:6], buildings[i] >= min_num_building_per_1kPeople[i] * population)
@constraint(m, dot(cost,buildings) <= budgets)
@objective(m, Max,buildings'*(profits + attractiveness))

s = solve(m)
println()
# get the values of the number of buildings vector
building = getvalue(buildings)
for b = 1:6
    println("Number of ",building_types[b]," to build: ", building[b])
end
println("At the budgets of \$", budgets, " the objective would be ", getobjectivevalue(m))

Academic license - for non-commercial use only

Number of business to build: 50.0
Number of hospital to build: 10.0
Number of park to build: 30.0
Number of farm to build: 10.0
Number of school to build: 10.0
Number of home to build: 18225.0
At the budgets of $10000000 the objective would be 55375.0


## 3.B. Second Model With Layout Mapping: Zonning

In [84]:
# data
building_types = ["biz", "hosp", "park", "farm", "school", "home"]
num_types = length(building_types)
# we converted profits, and attractiveness on a scale from -10 to 10
cost = [20,100,30,20,60,4]
profits = [9,-3,-1,6,-4,1]
attractiveness = [4,-1,3,-5,6,2]
min_num_building_per_1kPeople = [1,1,1,1,1,1]
N = 6
numType = size(building_types,1)
# Base city
base = zeros(N,N,length(building_types))
base[3,2,:] = [0 1 0 0 0 0]
base[3,3,:] = [1 0 0 0 0 0]
base[3,4,:] = [0 0 0 0 0 1]
base[4,2,:] = [0 0 1 0 0 0]
base[4,3,:] = [0 0 0 1 0 0]
base[4,4,:] = [0 0 0 0 1 0]
base[1,:,:] = zeros(N,length(building_types))
base[6,:,:] = zeros(N,length(building_types))
base[:,1,:] = zeros(N,length(building_types))
base[:,6,:] = zeros(N,length(building_types))
function printMap(map, N, M, P)
    for i =1:N
       for j = 1:M
           foundType = false
            for k = 1:P
                if(map[i,j,k] == 1)
                    print(building_types[k])
                    print("\t")
                    foundType = true
                end
            end
            if foundType == false
                print("____\t")
            end
        end
        print("\n")
    end
end


using JuMP, Gurobi
# assume that we have population of 10,000 people
population = 10
# assume that we have a budget of
budgets = 1000
R = 100

m = Model(solver = GurobiSolver(OutputFlag = 0))
@variable(m, x[1:N, 1:N, 1:numType], Bin)

# can't have more than one building in a spot (D)
@constraint(m, flow[i in 1:N, j in 1:N], sum(x[i,j,:]) <= 1)

# buildings in the base must stay
@constraint(m, flow1[i in 1:N, j in 1:N, k in 1:numType], x[i,j,k] >= base[i,j,k])
@constraint(m, topSideZero, x[1,:,:] .== base[1,:,:])
@constraint(m, leftSideZero, x[:,1,:] .== base[:,1,:])
@constraint(m, bottomSideZero, x[6,:,:] .== base[6,:,:])
@constraint(m, rightSideZero, x[:,6,:] .== base[:,6,:])

# meet the minimum number per building
@constraint(m, flow2[i in 1:numType], sum(x[:,:,i]) >= min_num_building_per_1kPeople[i])

# Stay within the given budget (C)
@constraint(m, sum((sum(x[:,:,k]) - sum(base[:,:,k]))*cost[k] for k in 1:numType) <= budgets)

# Continuity Constraint (E)
@constraint(m, flow3[i in 2:N-1, j in 2:N-1, k in 1:numType], x[i,j,k] <=
    x[i-1,j,k] + x[i,j-1,k] + x[i+1,j,k] + x[i,j+1,k])

# Area:Perimeter Constraint (F)
@constraint(m, flow4[t in 1:numType],
    sum( sum( 4x[i,j,t] - x[i-1,j,t] - x[i,j-1,t] - x[i+1,j,t] - x[i,j+1,t] for j in 2:(N-1)) for i in 2:(N-1))
    <= R * sum( sum( x[i,j,t] for j in 2:N-1) for i in 2:(N-1)))

# Objective
@expression(m, totalProfit, sum( sum(x[:,:,k])*profits[k] for k in 1:numType))
@expression(m, totalAttractiveness, sum(sum(x[:,:,k])*attractiveness[k] for k in 1:numType))
@objective(m, Max, totalProfit + totalAttractiveness)

println(solve(m))
printMap(getvalue(x), N,N,numType)
getGraphMatrix(getvalue(x))

Academic license - for non-commercial use only
Optimal
____	____	____	____	____	____	
____	hosp	biz	home	biz	____	
____	hosp	biz	home	biz	____	
____	park	farm	school	biz	____	
____	park	farm	school	biz	____	
____	____	____	____	____	____	


## 3.C. Third Model With Layout Mapping and Year Iteration

## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned |\$1600 |
| col 2 is      | centered      |  \$12 |
| zebra stripes | are neat      |   \$1 |

### 4.A. Feel free to add subsections

#### 4.A.a. or subsubsections

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.